* Through exploratory data analysis, this paper preliminarily understands the data,and then establishes a logistic regression model to explore the important factors related to customer churn;Finally predicts the probability of customer churn.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data=pd.read_csv('/kaggle/input/Churn_Modelling.csv')

## understand the data

In [ ]:
data.info()

In [ ]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [ ]:
data.head()

* The rough distribution of numerical variables is described as follows.

* Firstly, we discuss the last line of the figure. Because the Exited variable is 0 or 1, the average value reflects the proportion of customers who have already sold out in the data given by the bank. That is, about 20% of the 10,000 samples have been sold. This proportion is basically consistent with the retail sales rate of commercial banks in the background introduction and is acceptable. 
* Active and inactive customers in IsActive Members are close to 1:1. 
* And 70% of customers own credit cards. Then we discuss the continuous variable Balance, whose mean median is quite different and the mean is less than the median, which may be left-biased distribution. 
* Other data are well distributed.

In [ ]:
data.describe().T

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.sans-serif']=['SimHei']
fig = plt.figure(figsize=(12,8))
plt.title('Sample Label Number Statistics')
ax1 = fig.add_subplot(121)
sns.countplot(data['Exited'])
ax2 = fig.add_subplot(122)
data['Exited'].value_counts().plot(kind='pie',autopct='%.2f')
plt.show()

* Discuss the impact of customer credit score on retailers. The classified boxes are as follows. It shows the distribution relationship between the customer credit score and whether or not the seller.

* my primary understading about this result is that
> The median credit score of the retailer customer is slightly lower than that of the normal customer, and the overall difference is not significant. However, it is necessary to pay attention to the outlier value in the credit score of the customer, so when the bank maintains the customer, the user with lower credit score, especially the customer with abnormal credit, needs to reduce the investment, and such customers are likely to sell.

In [ ]:
sns.boxplot(y='CreditScore',x ='Exited',data=data)
plt.show()

In [ ]:
def my_plot(data,col,figsize=(8,5),yticks=np.arange(0,1,.1)):
    positive = data.loc[data['Exited'] == 1]
    exited_rate = (positive[col].value_counts() / data[col].value_counts()).sort_index().fillna(0)
    exited_numbers = data.groupby([col, 'Exited']).size().reset_index().pivot(columns='Exited', index=col, values=0).reindex(index=exited_rate.index).fillna(0)
    #fig = plt.figure(figsize=figsize)
    ax1 = fig.add_subplot(111)
    exited_numbers.plot(kind='bar',stacked=True,ax=ax1)
    ax2 = ax1.twinx()
    ax2.plot(exited_rate.index.map(lambda x: str(x)),exited_rate,'og--')
    ax2.set_yticks(yticks)
    plt.show()

In [ ]:
positive = data.loc[data['Exited'] == 1]
exited_rate = (positive['Age'].value_counts() / data['Age'].value_counts()).sort_index().fillna(0)
exited_numbers = data.groupby(['Age', 'Exited']).size().reset_index().pivot(columns='Exited', index='Age', values=0).reindex(index=exited_rate.index).fillna(0)
fig = plt.figure(figsize=(10,5))
ax1 = fig.add_subplot(111)
exited_numbers.plot(kind='bar',stacked=True,ax=ax1)
ax2 = ax1.twinx()
#ax2.plot(exited_rate.index.map(lambda x: str(x)),exited_rate,'og--') 
## In my local IDE, I can run this line of code smoothly. But kaggle will report errors
plt.show()

** Generally speaking, the longer a bank deals with business, the less likely it is that the seller will leave.

However, in the discount chart of the customer's service life and the sales rate, the longer the sales time is, the less obvious it is.

The lower the household rate is. It is worth noting that the highest customer sales rate is less than one year, which is in line with.

It is expected that some customers will choose to open accounts for certain business needs, but they will not remain for a long time.

The bank. So on the other hand, how to develop new customers into resident active users is what banks should pay attention to.

. In addition, the customer sales rate of opening accounts for 7 years is low, but that of retaining accounts for 8 years and 9 years is rising.

Gao, indicating that 7 years may be a key node for customers.

* The diagrams between customer account balances and turnover exceeded expectations. Most of the retired users have no account balance of 0, while more than 25% of the normal users have zero account balance. Analyzing the reasons, in the case of real data sources, it may be that in the transnational business of the bank, some customers who have consumed have not chosen to deal with the cancellation when they return home, while those who still have a large balance have to choose the cancellation. Another possible explanation is that customers with large balances find that the bank's products are not cost-effective and choose to sell, while those with no balances do not care whether they are good for themselves, which needs further study from the product perspective.

In [ ]:
sns.boxplot(data=data,x='Exited',y='Balance')
plt.show()

* Customers with 1 product number accounted for 69% of the total customers. Among all customers who own one product, 27% are retailers, slightly higher than the average. Only 7.5% of customers own two products. The sales rate of customers with three products and four products is 82.7% and 100% respectively. It can be seen that customers with more than three products are likely to deal with sales, while those with two products are relatively stable groups. Customer poles holding more than 3 products while maintaining customers

* Its instability, or the choice to give up, will be limited resources into the holding of products for 1 customers, may have better returns.

In [ ]:
data['NumOfProducts'].value_counts()
positive = data.loc[data['Exited'] == 1]
negative = data.loc[data['Exited'] == 0]
data[['NumOfProducts','Exited']].pivot_table(index='NumOfProducts',columns='Exited',aggfunc=len,margins=True).fillna(0)

*  The following results can be obtained: Among the users with credit cards, 79.8% have no dealers, and 79.1% have no credit cards and no dealers. This shows that the bank's credit cards seem to have little effect on retaining customers. In combination with the bank's commercial banks for transnational business, even if they use credit cards to retain customers. Households may not work. Among the cancelled customers, 69% have credit cards. Again, the bank's reference card business is not a good choice for consumers.

In [ ]:
data[['HasCrCard','Exited']].pivot_table(index='Exited',columns='HasCrCard',aggfunc=len,margins=True)

* Fourteen percent of users considered to be active, compared with nearly one fourth of non-active users. That is to say, the sales rate of active users has dropped significantly. This indicator may be extremely important for data modelling and should be noted in subsequent modelling.

In [ ]:
data[['IsActiveMember','Exited']].pivot_table(index='Exited',columns='IsActiveMember',aggfunc=len,margins=True)

* Customer's wage level may affect whether they go to the bank to handle business, and the box chart of the wage level of the seller and non-seller customers is almost the same, that is, the wage level has little effect on whether the bank customer is the seller or not.

In [ ]:
sns.boxplot(x='Exited',y='EstimatedSalary',data=data)
plt.show()

## Logistic model

In [ ]:
import statsmodels.api as sm

In [ ]:
data=data.replace({'Female':0,'Male':1})

In [ ]:
X_train = data.drop('Exited',axis=1)
y = data['Exited']
X_train.shape,y.shape

In [ ]:
log = sm.Logit(y,pd.get_dummies(X_train))
result=log.fit()

* The results show that the P value of the model is less than 0.05, which indicates that the logistic regression model is of statistical significance for the data set modeling. When the coefficient of the model is tested, the three variables of customer's country/region, customer's estimated wage and whether the customer owns credit card are not tested. Although the customer's P value is greater than 0.05, the difference is not large. We can not completely deny its impact on the model. Therefore, we only delete three variables whose P value is much greater than 0.05 and re-model them.

In [ ]:
log = sm.Logit(y,pd.get_dummies(X_train))

In [ ]:
print(result.summary2())

In [ ]:
X_train_2 = data.drop(['Exited','Tenure','EstimatedSalary','HasCrCard'],axis=1)

In [ ]:
log_2 = sm.Logit(y,pd.get_dummies(X_train_2))
result_2 = log_2.fit()

In [ ]:
print(result_2.summary2())

* The results of quadratic modeling show that all variables coefficients and models have passed the test. That is to say, the final model is:
```math
log [p(y=1))/p(y=0)]=-0.041*Credictscore-0.645*Gender+0.0564*Age−0.307*Numofproducts−1.0649*IsActivtMember
```

* Creditscore's maximum likelihood estimator is negative. That is to say, the probability of customer churn is inversely proportional to credit score while other conditions remain unchanged. The higher the credit, the smaller the probability of customer churn. When credit score increases by 1 point, the probability of retailers changing 0.0041 = 0.99 < 1, that is to say, changing the credit score of a unit tends to be more non-retailers, which is consistent with the results of exploratory analysis.

* The coefficients of variables are also negative, that is to say, when other conditions remain unchanged, the probability of female customers changing to male customers is higher than that of male customers. Under the same conditions, the probability of female customers changing to male customers is lower than that of male customers e^0.654 = 0.51.

* Age's coefficient is positive, that is, the older the age, the higher the probability of retailers. The results are consistent with those of univariate analysis. Keeping other conditions unchanged, the probability of selling cards increases by 0.0564 = 1.05 > 1 with age increasing by 1 year, that is, the increase of age unit will lead to customers'more inclination to sell cards.

* The coefficient is negative, that is to say, the more the number of products owned, the lower the probability of retailers.

* The coefficients are negative, that is to say, the probability of active member retailers is lower while other conditions remain unchanged. The above analysis is consistent with the results of exploratory analysis, which shows that univariate analysis is also necessary and can enhance the understanding of data.

## predict

In [ ]:
pred= result_2.predict(pd.get_dummies(X_train_2))

In [ ]:
yhat = 1*(pred>0.20)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
print(confusion_matrix(np.array(data['Exited']),np.array(yhat),labels=[0,1]))

* Based on the above model, the probability of customer churn is predicted, and the threshold value is 20% of the prior probability. The customer is divided into seller or non-seller customers. The predicted confusion matrix is shown in the following figure. The probability of misjudgement is 29.9%, but the TPR value is 70.0%. That is to say, 70% of the customers predicted to be retailers are correct, which can improve the difficulty of maintaining customers.